In [1]:
# https://docs.bittensor.com/subnets/register-validate-mine#check-the-permit-status
import bittensor as bt
import asyncio
from prompting.protocol import StreamPromptingSynapse
from prompting.forward import handle_response
from transformers import AutoTokenizer

# NET_UID = 61
NET_UID = 170
NETWORK = "test"
WALLET = "validator"
# WALLET = "miner"
wallet = bt.wallet(name=WALLET)
subtensor = bt.subtensor(network=NETWORK)
subnet = subtensor.metagraph(netuid=NET_UID)

hotkey = wallet.hotkey.ss58_address
my_uid = subnet.hotkeys.index(wallet.hotkey.ss58_address)
sub = bt.subtensor(NETWORK)
mg = sub.metagraph(NET_UID)
if hotkey not in mg.hotkeys:
    print(f"Hotkey {hotkey} deregistered")
else:
    print(f"Hotkey {hotkey} is registered. UID: {my_uid}")
active_uids = subnet.uids.tolist()
print(f"Active UIDs (total: {len(active_uids)}): {active_uids}")
print(f"My hotkey is active: {my_uid in active_uids}")
print(f"Is val permit: {subnet.validator_permit[my_uid]}")
print(f"1000+ TAO:")

/workspace/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-05 10:59:13,583	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
INFO:bittensor: - Connected to test network and wss://test.finney.opentensor.ai:443/. - 


2024-07-05 10:59:30.694 |       INFO       |  - Connected to test network and wss://test.finney.opentensor.ai:443/. - 


INFO:bittensor: - Connected to test network and wss://test.finney.opentensor.ai:443/. - 


2024-07-05 10:59:33.646 |       INFO       |  - Connected to test network and wss://test.finney.opentensor.ai:443/. - 
Hotkey 5Fk35HgrTqqUffK7WN8FG4euZ8MpKx35mUYz9kgwj3UDnNHr is registered. UID: 0
Active UIDs (total: 2): [0, 1]
My hotkey is active: True
Is val permit: True
1000+ TAO:


In [38]:
uids = [my_uid]
axons = [subnet.axons[uid] for uid in uids]

# Directly call dendrite and process responses in parallel
dendrite = bt.dendrite(wallet=wallet)
roles = ["user"]
messages = ["Organic prompt query. Question: Capital of Zimbabwe?"]
timeout = 40
model_name = "casperhansen/llama-3-8b-instruct-awq"
tokenizer = AutoTokenizer.from_pretrained(model_name)

streams_responses = await dendrite(
    axons=axons,
    synapse=StreamPromptingSynapse(roles=roles, messages=messages),
    timeout=timeout,
    deserialize=False,
    streaming=True,
)
stream_results_dict = dict(zip(uids, streams_responses))
handle_stream_responses_task = asyncio.create_task(handle_response(stream_results_dict, tokenizer))
stream_results = await handle_stream_responses_task

bt.logging.info(f"Response:\n {stream_results[0].synapse.completion}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:bittensor: - Response:
 The capital of Zimbabwe is Harare. - 


In [17]:
# Sample validators.
# TODO: Stress test: how many queries

uids = [my_uid]
axons = [subnet.axons[uid] for uid in uids]

# Directly call dendrite and process responses in parallel
dendrite = bt.dendrite(wallet=wallet)
roles = ["user"]
messages = ["Organic prompt query. Question: Capital of Zimbabwe?"]
timeout = 40
model_name = "casperhansen/llama-3-8b-instruct-awq"
tokenizer = AutoTokenizer.from_pretrained(model_name)

streams_responses = await dendrite(
    axons=axons,
    synapse=StreamPromptingSynapse(roles=roles, messages=messages),
    timeout=timeout,
    deserialize=False,
    streaming=True,
)

stream_results_dict = dict(zip(uids, streams_responses))
handle_stream_responses_task = asyncio.create_task(handle_response(stream_results_dict, tokenizer))
stream_results = await handle_stream_responses_task

bt.logging.info(f"Response:\n {stream_results[0].synapse.completion}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:bittensor: - Response:
  - 


In [63]:
a = await asyncio.create_task(handle_response(stream_results_dict, tokenizer))

ERROR:bittensor: - Error in generating reference or handling responses for uid 0: Something went wrong with miner uid 0, Synapse is not StreamPromptingSynapse.
Traceback (most recent call last):
  File "/workspace/prompting-organic/prompting/forward.py", line 74, in process_stream
    raise ValueError(
ValueError: Something went wrong with miner uid 0, Synapse is not StreamPromptingSynapse.
 - 


In [64]:
a

[SynapseStreamResult(exception=ValueError('Something went wrong with miner uid 0, Synapse is not StreamPromptingSynapse.'), uid=0, accumulated_chunks=[], accumulated_chunks_timings=[], tokens_per_chunk=[], synapse=StreamPromptingSynapse(required_hash_fields=['messages'], roles=['user'], messages=['failure'], completion=''))]

In [62]:
stream_results

[SynapseStreamResult(exception=None, uid=0, accumulated_chunks=[], accumulated_chunks_timings=[], tokens_per_chunk=[], synapse=StreamPromptingSynapse(required_hash_fields=['messages'], roles=['user'], messages=['Organic prompt query. Question: Capital of Zimbabwe?'], completion=''))]

In [19]:
# TODO: Stress test: how many queries
from prompting.protocol import StreamPromptingSynapse
from prompting.forward import handle_response
from transformers import AutoTokenizer

uids = [my_uid]
axons = [subnet.axons[uid] for uid in uids]

# Directly call dendrite and process responses in parallel
dendrite = bt.dendrite(wallet=wallet)
roles = []
messages = []
timeout = 40
model_name = "casperhansen/llama-3-8b-instruct-awq"
tokenizer = AutoTokenizer.from_pretrained(model_name)
while True:
    message = input()
    if message == "exit":
        break
    roles.append("user")
    messages.append(message)
    streams_responses = await dendrite(
        axons=axons,
        synapse=StreamPromptingSynapse(roles=roles, messages=messages),
        timeout=timeout,
        deserialize=False,
        streaming=True,
    )
    stream_results_dict = dict(zip(uids, streams_responses))
    stream_results = await asyncio.create_task(handle_response(stream_results_dict, tokenizer))
    response = stream_results[0].synapse.completion

    print(f"User: {message}")
    print(f"Assistant: {response}")

    messages.append(response)
    roles.append("assistant")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


User: Hey Im John
Assistant: Hi John! How can I assist you today?


In [ ]:
stream_results[0].synapse.completion

In [ ]:
import asyncio
import concurrent.futures
import time

async def val():
    print("Val !")
    await asyncio.sleep(5)

async def forward():
    while True:
        print("Forward")
        await asyncio.sleep(1)

async def background_task():
    while True:
        print("Background")
        await val()
        await asyncio.sleep(1)

loop = asyncio.get_event_loop()
try:
    loop.run_forever()
except RuntimeError:
    pass

task = loop.create_task(forward())
task2 = loop.create_task(background_task())
try:
    await asyncio.wait_for(task, timeout=5)
except asyncio.TimeoutError:
    print("Forward task timed out")

In [ ]:
import asyncio
import concurrent.futures
import time

# Asynchronous background task
async def background_task():
    while True:
        print("Background task is running")
        await asyncio.sleep(1)

# Function to run the asyncio event loop in a separate thread
def start_asyncio_loop(loop):
    asyncio.set_event_loop(loop)
    loop.run_forever()

def main():
    # Create a new asyncio event loop
    loop = asyncio.new_event_loop()

    # Start the event loop in a new thread
    executor = concurrent.futures.ThreadPoolExecutor()
    executor.submit(start_asyncio_loop, loop)

    # Schedule the asynchronous background task
    asyncio.run_coroutine_threadsafe(background_task(), loop)

    # Synchronous while True loop in main
    while True:
        print("Main loop is running")
        time.sleep(1)

main()